In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# read the ratings data
ratings = pd.read_csv('ratings 23-24.csv')

# read the kaggle id data
kaggle = pd.read_csv('kaggle.csv')

# read the ncaa stats data
stats = pd.read_csv('ncaa stats 23-24.csv')

# keep only the rows that have a team name with NCAA in it
# stats = stats[stats['School'].str.contains('NCAA')]

# rename the Schools to get rid of the NCAA at the end
stats['School'] = stats['School'].str.replace(' NCAA', '').str.strip()

#for column in ['MP', 'OMP', '3PA', 'O3PA', 'OFG', 'TRB', 'OTRB', 'FG', 'OFG', 'FGA', 'PF', 'OPF', 'OFGA', '3P', 'OORB', 'OAST', 'AST', 'FT', 'OFT', 'FTA', 'OFTA', 'OFGA', 'ORB', 'OTRB', 'TOV', 'OTOV', 'STL', 'BLK', 'OSTL', 'OBLK']:
for column in ['FGA', 'OTRB', 'TOV']:
  stats[column] = stats[column] / stats['TG']

# drop unnamed columns
ratings = ratings.loc[:, ~ratings.columns.str.contains('^Unnamed')]

# drop bad columns
ratings = ratings.drop(['Rk', 'Conf'], axis=1)

# rename Pts column to PPG
ratings = ratings.rename(columns={'Pts': 'PPG'})

# rename Opp column to OPPG
ratings = ratings.rename(columns={'Opp': 'OPPG'})

# rename the ORtg column to AdjORtg, DRtg to AdjDRtg, and NRtg to AdjNRtg
ratings = ratings.rename(columns={'ORtg': 'AdjORtg', 'DRtg': 'AdjDRtg', 'NRtg': 'AdjNRtg'})

# create a dictionary between the team name and the kaggle id
kaggle_dict = {}
for index, row in kaggle.iterrows():
    kaggle_dict[row['team name ']] = row['TeamID']

# create a new column in the dataframe with the kaggle id using the dictionary and the team name
ratings['kaggle_id'] = ratings['School'].map(kaggle_dict)

# merge the ratings data with the ncaa stats data
ratings = ratings.merge(stats, left_on='School', right_on='School')

# make kaggle_id the third column
cols = list(ratings.columns)
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
ratings = ratings[cols]

# standardize all the columns except for the team name and kaggle id with z-scores
# for col in cols:
#     if col not in ['School', 'kaggle_id']:
#         ratings[col] = (ratings[col] - ratings[col].mean()) / ratings[col].std()

# save to a data file
ratings.to_csv('data.csv', index=False)

In [12]:
# read the stein file
stein = pd.read_csv('stein.csv')

# read the data file
data = pd.read_csv('data.csv')

# merge the stein data with the data file using the kaggle id
data = data.merge(stein, left_on='kaggle_id', right_on='kaggle_id')

# save to a data file
data.to_csv('data-new.csv', index=False)